# GP-VAE Training on COIL-100 (Interpolation Task) - Periodic Kernel

This notebook trains **GP-VAE** on COIL-100 dataset using the **Periodic kernel** for the **interpolation task**.

## Task: Interpolation
- **Train**: 9 views at even indices (0°, 40°, 80°, 120°, 160°, 200°, 240°, 280°, 320°)
- **Val**: 5 views at odd indices (20°, 100°, 180°, 260°, 340°)
- **Test**: 4 views (60°, 140°, 220°, 300°)
- **Goal**: Predict unseen intermediate views using GP interpolation

## Kernel: Periodic
- **Standard periodic kernel** for exact periodic patterns
- k(θ, θ') = variance × exp(-2 × sin²(π|θ-θ'|/period) / lengthscale²)
- **period=360°** fixed for full rotation
- **Parameters**: 2 learnable (lengthscale, variance)
- **Best for**: Data with exact periodicity (360° = 0°)

## Dataset Info:
- **COIL-100**: 100 objects × 18 views (every 20°: 0°, 20°, ..., 340°)
- **Image size**: 128×128×3 RGB

## Prerequisites:
- ✅ Trained VAE weights from `train_vae_colab_interpolation.ipynb`
- ✅ COIL-100 data file: `data/coil100/coil100_task2_interpolation.h5`

## 1. Check GPU Availability

In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: GPU not detected!")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU Device: Tesla T4
GPU Memory: 15.83 GB


## 2. Auto-Detect Project Path

In [2]:
import os
import sys

current_dir = os.getcwd()
print(f"📍 Current directory: {current_dir}")

# Task configuration
DATA_TASK = "task2_interpolation"
KERNEL_TYPE = "periodic"

if current_dir == '/content':
    print("\n🔄 Mounting Google Drive...")
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        drive_path = '/content/drive/MyDrive/gppvae'
        if os.path.exists(drive_path):
            PROJECT_PATH = drive_path
            print(f"✅ Found project in Google Drive: {PROJECT_PATH}")
        else:
            print(f"⚠️ Project not found at: {drive_path}")
            PROJECT_PATH = '/content'
    except Exception as e:
        print(f"Could not mount Drive: {e}")
        PROJECT_PATH = '/content'
else:
    if 'notebooks' in current_dir:
        PROJECT_PATH = os.path.dirname(current_dir)
    else:
        PROJECT_PATH = current_dir
    print(f"💻 Using project path: {PROJECT_PATH}")

# Check required files
print(f"\n🔍 Checking required files:")
data_path = os.path.join(PROJECT_PATH, f'data/coil100/coil100_{DATA_TASK}.h5')
required = {
    'GPPVAE code': os.path.exists(os.path.join(PROJECT_PATH, 'GPPVAE')),
    'COIL-100 data': os.path.exists(data_path),
}

# Look for VAE weights trained on interpolation task
vae_base_dir = os.path.join(PROJECT_PATH, f'out/vae_colab_{DATA_TASK}')
vae_run_found = False
if os.path.exists(vae_base_dir):
    runs = [d for d in os.listdir(vae_base_dir) if os.path.isdir(os.path.join(vae_base_dir, d))]
    for run in runs:
        weights_dir = os.path.join(vae_base_dir, run, 'weights')
        if os.path.exists(weights_dir) and any(f.endswith('.pt') for f in os.listdir(weights_dir)):
            vae_run_found = True
            break
required['VAE weights'] = vae_run_found

for name, exists in required.items():
    status = "✅" if exists else "❌"
    print(f"   {status} {name}")

print(f"\n📊 Interpolation Task Info:")
print(f"   Train: 9 views (even indices) × 100 objects = 900 samples")
print(f"   Val: 5 views (odd indices) × 100 objects = 500 samples")
print(f"   Test: 4 views × 100 objects = 400 samples")

📍 Current directory: /content

🔄 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Found project in Google Drive: /content/drive/MyDrive/gppvae

🔍 Checking required files:
   ✅ GPPVAE code
   ✅ COIL-100 data
   ✅ VAE weights

📊 Interpolation Task Info:
   Train: 9 views (even indices) × 100 objects = 900 samples
   Val: 5 views (odd indices) × 100 objects = 500 samples
   Test: 4 views × 100 objects = 400 samples


## 3. Install Dependencies

In [3]:
!pip install -q wandb==0.12.21 imageio==2.15.0 pyyaml

import wandb
import numpy as np
print("✅ All dependencies installed!")

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
✅ All dependencies installed!


## 4. Login to W&B (Optional)

In [4]:
import wandb
wandb.login()

wandb: Currently logged in as: minh1008 (minh1008-ludwig-maximilianuniversity-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## 5. Navigate to Project

In [6]:
import os
import sys

os.chdir(PROJECT_PATH)
print(f"Current directory: {os.getcwd()}")

sys.path.insert(0, os.path.join(PROJECT_PATH, 'GPPVAE/pysrc/coil100'))

Current directory: /content/drive/MyDrive/gppvae


## 6. Find VAE Weights

In [7]:
import os
import pickle

# Look for VAE trained on interpolation task
vae_base_dir = f'./out/vae_colab_{DATA_TASK}'
vae_runs = []

if os.path.exists(vae_base_dir):
    for run_dir in sorted(os.listdir(vae_base_dir), reverse=True):
        run_path = os.path.join(vae_base_dir, run_dir)
        cfg_path = os.path.join(run_path, 'vae.cfg.p')
        weights_dir = os.path.join(run_path, 'weights')

        if os.path.exists(cfg_path) and os.path.exists(weights_dir):
            weight_files = sorted([f for f in os.listdir(weights_dir) if f.endswith('.pt')])
            if weight_files:
                vae_runs.append({
                    'run_dir': run_dir,
                    'cfg_path': cfg_path,
                    'weights_dir': weights_dir,
                    'weight_files': weight_files
                })

if vae_runs:
    print(f"✅ Found {len(vae_runs)} VAE run(s) for interpolation task")
    latest = vae_runs[0]
    print(f"\n💡 Latest: {latest['run_dir']}")
    print(f"   VAE_CFG = '{latest['cfg_path']}'")
    print(f"   VAE_WEIGHTS = '{os.path.join(latest['weights_dir'], latest['weight_files'][-1])}'")
else:
    print("❌ No VAE runs found for interpolation task!")
    print("   Run train_vae_colab_interpolation.ipynb first.")

✅ Found 1 VAE run(s) for interpolation task

💡 Latest: 20260119_135537
   VAE_CFG = './out/vae_colab_task2_interpolation/20260119_135537/vae.cfg.p'
   VAE_WEIGHTS = './out/vae_colab_task2_interpolation/20260119_135537/weights/weights.00499.pt'


## 7. Configure Training

**Periodic Kernel Parameters:**
- `period`: Fixed at 360° for full rotation
- `lengthscale`: Controls smoothness within one period. **Use ~1.0 for proper interpolation**
- `variance`: Signal variance. Default: 1.0

**⚠️ Important:** For the interpolation task, the lengthscale is critical!
- `lengthscale=30` → all views 99.9% correlated (no interpolation structure)
- `lengthscale=1.0` → proper correlation decay with angle difference

In [12]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# ============================================================================
# UPDATE THESE PATHS!
# ============================================================================
VAE_CFG = './out/vae_colab_task2_interpolation/20260119_135537/vae.cfg.p'
VAE_WEIGHTS = './out/vae_colab_task2_interpolation/20260119_135537/weights/weights.00499.pt'

CONFIG = {
    # Data
    'data': f'./data/coil100/coil100_{DATA_TASK}.h5',
    'outdir': f'./out/gppvae_coil100_{KERNEL_TYPE}_{DATA_TASK}/{timestamp}',

    # VAE
    'vae_cfg': VAE_CFG,
    'vae_weights': VAE_WEIGHTS,

    # Training
    'epochs': 1500,
    'batch_size': 64,
    'vae_lr': 0.001,
    'gp_lr': 0.001,
    'xdim': 64,

    # Kernel - Periodic (IMPORTANT: use proper lengthscale for interpolation!)
    'view_kernel': KERNEL_TYPE,
    'kernel_kwargs': {
        'period': 360.0,      # Fixed: full rotation
        'lengthscale': 1.0,   # Proper scale for interpolation (NOT 30!)
        'variance': 1.0,
    },

    # Logging
    'epoch_cb': 100,
    'use_wandb': True,
    'wandb_project': 'gppvae-coil100',
    'wandb_run_name': f'gppvae_{KERNEL_TYPE}_{DATA_TASK}_{timestamp}',
    'seed': 0,
}

print("GP-VAE Training Configuration (Interpolation Task):")
print("=" * 60)
for key, value in CONFIG.items():
    print(f"  {key:20s}: {value}")
print("=" * 60)

if not os.path.exists(CONFIG['vae_weights']):
    print(f"\n⚠️ Update VAE_CFG and VAE_WEIGHTS paths!")

GP-VAE Training Configuration (Interpolation Task):
  data                : ./data/coil100/coil100_task2_interpolation.h5
  outdir              : ./out/gppvae_coil100_periodic_task2_interpolation/20260119_180211
  vae_cfg             : ./out/vae_colab_task2_interpolation/20260119_135537/vae.cfg.p
  vae_weights         : ./out/vae_colab_task2_interpolation/20260119_135537/weights/weights.00499.pt
  epochs              : 1500
  batch_size          : 64
  vae_lr              : 0.001
  gp_lr               : 0.001
  xdim                : 64
  view_kernel         : periodic
  kernel_kwargs       : {'period': 360.0, 'lengthscale': 1.0, 'variance': 1.0}
  epoch_cb            : 100
  use_wandb           : True
  wandb_project       : gppvae-coil100
  wandb_run_name      : gppvae_periodic_task2_interpolation_20260119_180211
  seed                : 0


## 8. Import Modules

In [13]:
# IMPORTANT: Add coil100 to path FIRST before importing anything
# This ensures coil100's data_parser is used, not faceplace's
import sys
import os

coil100_path = os.path.join(PROJECT_PATH, 'GPPVAE/pysrc/coil100')
sys.path.insert(0, coil100_path)    # Add coil100 first (so it has priority)

os.chdir(coil100_path)
print(f"Working directory: {os.getcwd()}")
print(f"sys.path priority: coil100 > faceplace")

import matplotlib
matplotlib.use('Agg')

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from vae import FaceVAE
from vmod import Vmodel
from gp import GP
import numpy as np
import logging
import pylab as pl
from utils import smartSum, smartAppendDict, smartAppend, export_scripts
from callbacks import callback_gppvae
import pickle
import time
import wandb

# COIL-100 data parser (explicitly import from coil100, not faceplace)
from data_parser import COIL100Dataset, get_n_views, get_num_objects

# Verify we're using the right data_parser
import data_parser
print(f"✅ data_parser loaded from: {data_parser.__file__}")
if 'coil100' in data_parser.__file__:
    print("✅ Using COIL-100 data_parser (correct!)")
else:
    print("❌ WARNING: Using faceplace data_parser (wrong!)")
print("✅ All modules imported successfully!")

Working directory: /content/drive/MyDrive/gppvae/GPPVAE/pysrc/coil100
sys.path priority: coil100 > faceplace
✅ data_parser loaded from: /content/drive/MyDrive/gppvae/GPPVAE/pysrc/coil100/data_parser.py
✅ Using COIL-100 data_parser (correct!)
✅ All modules imported successfully!


## 9. Setup Training Environment

In [14]:
os.chdir(PROJECT_PATH)

outdir = CONFIG['outdir']
wdir = os.path.join(outdir, "weights")
fdir = os.path.join(outdir, "plots")
os.makedirs(wdir, exist_ok=True)
os.makedirs(fdir, exist_ok=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

log_format = "%(asctime)s %(message)s"
logging.basicConfig(level=logging.INFO, format=log_format, datefmt="%m/%d %I:%M:%S %p")
fh = logging.FileHandler(os.path.join(outdir, "log.txt"))
fh.setFormatter(logging.Formatter(log_format))
logging.getLogger().addHandler(fh)

export_scripts(os.path.join(outdir, "scripts"))
print(f"✅ Output: {outdir}")

Using device: cuda:0
✅ Output: ./out/gppvae_coil100_periodic_task2_interpolation/20260119_180211


## 10. Initialize Models

**Interpolation Task Specifics:**
- Training views: indices [0, 2, 4, 6, 8, 10, 12, 14, 16] → angles [0°, 40°, 80°, ...]
- Validation views: indices [1, 5, 9, 13, 17] → angles [20°, 100°, 180°, ...]
- The GP must learn to predict intermediate views from training views

In [15]:
torch.manual_seed(CONFIG['seed'])

if CONFIG['use_wandb']:
    wandb.init(project=CONFIG['wandb_project'], name=CONFIG['wandb_run_name'], config=CONFIG)

# Load VAE
vae_cfg = pickle.load(open(CONFIG['vae_cfg'], "rb"))
vae = FaceVAE(**vae_cfg).to(device)
vae.load_state_dict(torch.load(CONFIG['vae_weights'], map_location=device))
print(f"✅ VAE loaded")

# Load data
train_data = COIL100Dataset(CONFIG['data'], split='train', use_angle_encoding=False)
val_data = COIL100Dataset(CONFIG['data'], split='val', use_angle_encoding=False)
train_queue = DataLoader(train_data, batch_size=CONFIG['batch_size'], shuffle=True)
val_queue = DataLoader(val_data, batch_size=CONFIG['batch_size'], shuffle=False)

# IMPORTANT: Use get_num_objects for correct P (includes all objects from all splits)
P = get_num_objects(CONFIG['data'])  # 100 for COIL-100
Q = get_n_views()  # 18 total views
print(f"P={P}, Q={Q}")

# Show interpolation task structure
train_views = sorted(train_data.Rid.unique().tolist())
val_views = sorted(val_data.Rid.unique().tolist())
print(f"\n📊 Interpolation Task:")
print(f"   Train views (Rid): {train_views} → angles {[v*20 for v in train_views]}°")
print(f"   Val views (Rid):   {val_views} → angles {[v*20 for v in val_views]}°")
print(f"   Train samples: {len(train_data)}, Val samples: {len(val_data)}")

# Create object and view tensors (Did and Rid are 1D tensors)
Dt = Variable(train_data.Did.long(), requires_grad=False).to(device)
Dv = Variable(val_data.Did.long(), requires_grad=False).to(device)
Wt = Variable(train_data.Rid.long(), requires_grad=False).to(device)
Wv = Variable(val_data.Rid.long(), requires_grad=False).to(device)

# Initialize Vmodel with Periodic kernel
print(f"\n🔬 Initializing '{KERNEL_TYPE}' kernel (period={CONFIG['kernel_kwargs']['period']}°)...")
vm = Vmodel(P=P, Q=Q, p=CONFIG['xdim'], view_kernel=CONFIG['view_kernel'], **CONFIG['kernel_kwargs']).to(device)
gp = GP(n_rand_effs=1).to(device)

# Show kernel matrix - important for interpolation!
K = vm.get_kernel_matrix()
print(f"\n📈 Kernel correlations (critical for interpolation):")
print(f"   K[0,0]={K[0,0].item():.4f} (self, 0°)")
print(f"   K[0,1]={K[0,1].item():.4f} (20° apart - val view from train)")
print(f"   K[0,2]={K[0,2].item():.4f} (40° apart - next train view)")
print(f"   K[0,9]={K[0,9].item():.4f} (180° apart)")

gp_params = nn.ParameterList()
gp_params.extend(vm.parameters())
gp_params.extend(gp.parameters())

vae_optimizer = optim.Adam(vae.parameters(), lr=CONFIG['vae_lr'])
gp_optimizer = optim.Adam(gp_params, lr=CONFIG['gp_lr'])
print(f"\n✅ Models initialized")

✅ VAE loaded
Built object ID mapping: 100 objects -> indices [0, 99]
Loaded COIL-100 train: 900 samples
  Y shape: torch.Size([900, 3, 128, 128])
  Unique objects: 100
  Did range: [0, 99] (remapped to contiguous)
  Angle encoding: indices [0, 17]
Loaded COIL-100 val: 500 samples
  Y shape: torch.Size([500, 3, 128, 128])
  Unique objects: 100
  Did range: [0, 99] (remapped to contiguous)
  Angle encoding: indices [0, 17]
P=100, Q=18

📊 Interpolation Task:
   Train views (Rid): [0, 2, 4, 6, 8, 10, 12, 14, 16] → angles [0, 40, 80, 120, 160, 200, 240, 280, 320]°
   Val views (Rid):   [1, 5, 9, 13, 17] → angles [20, 100, 180, 260, 340]°
   Train samples: 900, Val samples: 500

🔬 Initializing 'periodic' kernel (period=360.0°)...


/content/drive/MyDrive/gppvae/GPPVAE/pysrc/coil100/vmod.py:101: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).mH
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:1765.)
  L = torch.cholesky(K)



📈 Kernel correlations (critical for interpolation):
   K[0,0]=1.0000 (self, 0°)
   K[0,1]=0.9415 (20° apart - val view from train)
   K[0,2]=0.7914 (40° apart - next train view)
   K[0,9]=0.1353 (180° apart)

✅ Models initialized


## 11. Training Functions

In [ ]:
def encode_Y(vae, train_queue):
    vae.eval()
    with torch.no_grad():
        n = train_queue.dataset.Y.shape[0]
        Zm = Variable(torch.zeros(n, vae_cfg["zdim"]), requires_grad=False).to(device)
        Zs = Variable(torch.zeros(n, vae_cfg["zdim"]), requires_grad=False).to(device)
        for data in train_queue:
            y = data[0].to(device)
            idxs = data[-1].to(device)
            zm, zs = vae.encode(y)
            Zm[idxs], Zs[idxs] = zm.detach(), zs.detach()
    return Zm, Zs

def eval_step(vae, gp, vm, val_queue, Zm, Vt, Vv, Wv, val_Rid):
    """Evaluate GP-VAE on validation set (unseen views for interpolation task).
    
    Args:
        val_Rid: View indices for validation set (to compute per-view MSE)
    
    Returns:
        rv: Dict with mse_out, mse_val, and per-view MSE (mse_view_XXX)
    """
    rv = {}
    with torch.no_grad():
        _X = vm.x().data.cpu().numpy()
        _W = vm.v().data.cpu().numpy()
        covs = {"XX": np.dot(_X, _X.T), "WW": np.dot(_W, _W.T)}
        rv["vars"] = gp.get_vs().data.cpu().numpy()

        # GP prediction: predict validation latents from training latents
        vs = gp.get_vs()
        U, UBi, _ = gp.U_UBi_Shb([Vt], vs)
        Kiz = gp.solve(Zm, U, UBi, vs)
        # Zo: predicted latents for validation views using GP interpolation
        Zo = vs[0] * Vv.mm(Vt.transpose(0, 1).mm(Kiz))

        mse_out = Variable(torch.zeros(Vv.shape[0], 1), requires_grad=False).to(device)
        mse_val = Variable(torch.zeros(Vv.shape[0], 1), requires_grad=False).to(device)
        all_Yv, all_Yr, all_Yo = [], [], []

        for data in val_queue:
            idxs = data[-1].to(device)
            Yv = data[0].to(device)  # Ground truth validation images
            Zv = vae.encode(Yv)[0].detach()  # Encoded validation latents
            Yr = vae.decode(Zv)  # Reconstruction (encode-decode)
            Yo = vae.decode(Zo[idxs])  # GP-predicted images (interpolated!)

            # mse_out: How well can we predict UNSEEN views?
            mse_out[idxs] = ((Yv - Yo) ** 2).view(Yv.shape[0], -1).mean(1)[:, None].detach()
            mse_val[idxs] = ((Yv - Yr) ** 2).view(Yv.shape[0], -1).mean(1)[:, None].detach()
            all_Yv.append(Yv.data.cpu().numpy().transpose(0, 2, 3, 1))
            all_Yr.append(Yr.data.cpu().numpy().transpose(0, 2, 3, 1))
            all_Yo.append(Yo.data.cpu().numpy().transpose(0, 2, 3, 1))

        all_Yv = np.concatenate(all_Yv, axis=0)
        all_Yr = np.concatenate(all_Yr, axis=0)
        all_Yo = np.concatenate(all_Yo, axis=0)
        n_total = all_Yv.shape[0]
        sample_indices = np.arange(0, n_total, max(1, n_total // 24))[:24]
        imgs = {"Yv": all_Yv[sample_indices], "Yr": all_Yr[sample_indices], "Yo": all_Yo[sample_indices]}
        rv["mse_out"] = float(mse_out.data.mean().cpu())
        rv["mse_val"] = float(mse_val.data.mean().cpu())
        
        # Compute per-view MSE for interpolation analysis
        mse_out_cpu = mse_out.data.cpu().squeeze()
        val_Rid_cpu = val_Rid.cpu()
        unique_views = torch.unique(val_Rid_cpu).tolist()
        for view_idx in unique_views:
            mask = (val_Rid_cpu == view_idx)
            view_mse = mse_out_cpu[mask].mean().item()
            angle = int(view_idx * 20)  # Convert index to angle
            rv[f"mse_view_{angle:03d}"] = view_mse
        
    return rv, imgs, covs

def backprop_and_update(vae, gp, vm, train_queue, Dt, Wt, Eps, Zb, Vbs, vbs, vae_optimizer, gp_optimizer):
    rv = {}
    vae_optimizer.zero_grad()
    gp_optimizer.zero_grad()
    vae.train(); gp.train(); vm.train()

    for data in train_queue:
        y = data[0].to(device)
        eps = Eps[data[-1]]
        _d, _w = Dt[data[-1]], Wt[data[-1]]
        _Zb = Zb[data[-1]]
        _Vbs = [Vbs[0][data[-1]]]

        zm, zs = vae.encode(y)
        z = zm + zs * eps
        yr = vae.decode(z)
        recon_term, mse = vae.nll(y, yr)

        _Vs = [vm(_d, _w)]
        gp_nll_fo = gp.taylor_expansion(z, _Vs, _Zb, _Vbs, vbs) / vae.K
        pen_term = -0.5 * zs.sum(1)[:, None] / vae.K

        loss = (recon_term + gp_nll_fo + pen_term).sum()
        loss.backward()

        _n = train_queue.dataset.Y.shape[0]
        smartSum(rv, "mse", float(mse.data.sum().cpu()) / _n)
        smartSum(rv, "recon_term", float(recon_term.data.sum().cpu()) / _n)
        smartSum(rv, "pen_term", float(pen_term.data.sum().cpu()) / _n)

    vae_optimizer.step()
    gp_optimizer.step()
    return rv

print("✅ Training functions defined")

✅ Training functions defined


## 12. Train GP-VAE 🚀

**Key Metrics for Interpolation:**
- `mse_out`: MSE on **unseen views** (interpolated) - This is the key metric!
- `mse_val`: MSE on encode-decode (VAE reconstruction quality)
- `lengthscale`: Learned kernel lengthscale

In [ ]:
history = {}
start_time = time.time()

# Get validation view indices for per-view MSE tracking
val_Rid = val_data.Rid.to(device)
val_view_angles = sorted([int(v * 20) for v in val_data.Rid.unique().tolist()])
print(f"? Tracking per-view MSE for validation angles: {val_view_angles}°")

print(f"\n?🚀 Training GP-VAE with {KERNEL_TYPE} kernel for {CONFIG['epochs']} epochs...")
print(f"   Task: INTERPOLATION (predicting unseen intermediate views)")
print("=" * 70)

for epoch in range(CONFIG['epochs']):
    epoch_start = time.time()

    Zm, Zs = encode_Y(vae, train_queue)
    Eps = Variable(torch.randn(*Zs.shape), requires_grad=False).to(device)
    Z = Zm + Eps * Zs

    Vt = vm(Dt, Wt).detach()
    Vv = vm(Dv, Wv).detach()

    # Pass val_Rid for per-view MSE computation
    rv_eval, imgs, covs = eval_step(vae, gp, vm, val_queue, Zm, Vt, Vv, Wv, val_Rid)
    Zb, Vbs, vbs, gp_nll = gp.taylor_coeff(Z, [Vt])
    rv_eval["gp_nll"] = float(gp_nll.data.mean().cpu()) / vae.K

    rv_back = backprop_and_update(vae, gp, vm, train_queue, Dt, Wt, Eps, Zb, Vbs, vbs, vae_optimizer, gp_optimizer)
    rv_back["loss"] = rv_back["recon_term"] + rv_eval["gp_nll"] + rv_back["pen_term"]

    smartAppendDict(history, rv_eval)
    smartAppendDict(history, rv_back)
    smartAppend(history, "vs", gp.get_vs().data.cpu().numpy())

    epoch_time = time.time() - epoch_start
    vs = gp.get_vs().data.cpu().numpy()
    variance_ratio = vs[0] / (vs[0] + vs[1])

    # Get learned lengthscale
    learned_ls = vm.kernel.lengthscale.item()

    if epoch % 5 == 0 or epoch == CONFIG['epochs'] - 1:
        # Print per-view MSE summary
        view_mse_str = " | ".join([f"{a}°:{rv_eval[f'mse_view_{a:03d}']:.4f}" for a in val_view_angles])
        print(f"Epoch {epoch:4d} | MSE: {rv_back['mse']:.6f} | Interp: {rv_eval['mse_out']:.6f} | "
              f"GP NLL: {rv_eval['gp_nll']:.4f} | ℓ: {learned_ls:.2f} | v₀/(v₀+v₁): {variance_ratio:.3f}")
        print(f"         Per-view: {view_mse_str}")

    if CONFIG['use_wandb']:
        # Log basic metrics
        log_dict = {
            "epoch": epoch, "mse_train": rv_back["mse"], "mse_interp": rv_eval["mse_out"],
            "gp_nll": rv_eval["gp_nll"], "lengthscale": learned_ls, "variance_ratio": variance_ratio,
        }
        # Log per-view MSE
        for angle in val_view_angles:
            log_dict[f"mse_view_{angle:03d}"] = rv_eval[f"mse_view_{angle:03d}"]
        wandb.log(log_dict)

    if epoch % CONFIG['epoch_cb'] == 0 or epoch == CONFIG['epochs'] - 1:
        torch.save(vae.state_dict(), os.path.join(wdir, f"vae_weights.{epoch:05d}.pt"))
        torch.save({'gp_state': gp.state_dict(), 'vm_state': vm.state_dict()},
                   os.path.join(wdir, f"gp_weights.{epoch:05d}.pt"))
        ffile = os.path.join(fdir, f"plot.{epoch:05d}.png")
        callback_gppvae(epoch, history, covs, imgs, ffile)
        if CONFIG['use_wandb']:
            wandb.log({"reconstructions": wandb.Image(ffile)})
        print(f"  ✓ Checkpoint saved")

print(f"\n✅ Complete! Time: {(time.time()-start_time)/60:.1f}min")
print(f"   Final interpolation MSE: {rv_eval['mse_out']:.6f}")
print(f"   Learned lengthscale: {learned_ls:.2f}")
print(f"\n📊 Final per-view MSE:")
for angle in val_view_angles:
    print(f"   {angle:3d}°: {rv_eval[f'mse_view_{angle:03d}']:.6f}")
if CONFIG['use_wandb']: wandb.finish()

🚀 Training GP-VAE with periodic kernel for 1500 epochs...
   Task: INTERPOLATION (predicting unseen intermediate views)


/content/drive/MyDrive/gppvae/GPPVAE/pysrc/coil100/vae.py:46: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  x = F.upsample(x, scale_factor=2)


Epoch    0 | MSE: 0.002326 | Interp: 0.046617 | GP NLL: 0.0115 | ℓ: 1.00 | v₀/(v₀+v₁): 0.500
  ✓ Checkpoint saved


/content/drive/MyDrive/gppvae/GPPVAE/pysrc/coil100/vae.py:46: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  x = F.upsample(x, scale_factor=2)


Epoch    5 | MSE: 0.008273 | Interp: 0.045350 | GP NLL: 0.0137 | ℓ: 1.00 | v₀/(v₀+v₁): 0.497
Epoch   10 | MSE: 0.004539 | Interp: 0.037264 | GP NLL: 0.0128 | ℓ: 1.00 | v₀/(v₀+v₁): 0.495
Epoch   15 | MSE: 0.003405 | Interp: 0.030681 | GP NLL: 0.0112 | ℓ: 1.01 | v₀/(v₀+v₁): 0.492
Epoch   20 | MSE: 0.002854 | Interp: 0.027375 | GP NLL: 0.0093 | ℓ: 1.02 | v₀/(v₀+v₁): 0.490
Epoch   25 | MSE: 0.002402 | Interp: 0.026184 | GP NLL: 0.0092 | ℓ: 1.02 | v₀/(v₀+v₁): 0.488
Epoch   30 | MSE: 0.002214 | Interp: 0.024415 | GP NLL: 0.0091 | ℓ: 1.03 | v₀/(v₀+v₁): 0.486
Epoch   35 | MSE: 0.002067 | Interp: 0.023304 | GP NLL: 0.0081 | ℓ: 1.03 | v₀/(v₀+v₁): 0.484
Epoch   40 | MSE: 0.001946 | Interp: 0.022318 | GP NLL: 0.0079 | ℓ: 1.04 | v₀/(v₀+v₁): 0.482
Epoch   45 | MSE: 0.001864 | Interp: 0.021306 | GP NLL: 0.0075 | ℓ: 1.04 | v₀/(v₀+v₁): 0.480


## 13. Evaluate on Test Set (Optional)

Test set contains views [3, 7, 11, 15] → angles [60°, 140°, 220°, 300°]

In [ ]:
# Load test data
test_data = COIL100Dataset(CONFIG['data'], split='test', use_angle_encoding=False)
test_queue = DataLoader(test_data, batch_size=CONFIG['batch_size'], shuffle=False)

test_views = sorted(test_data.Rid.unique().tolist())
print(f"Test views (Rid): {test_views} → angles {[v*20 for v in test_views]}°")
print(f"Test samples: {len(test_data)}")

# Create test tensors
Dtest = Variable(test_data.Did.long(), requires_grad=False).to(device)
Wtest = Variable(test_data.Rid.long(), requires_grad=False).to(device)

# Evaluate
vae.eval()
vm.eval()
gp.eval()

with torch.no_grad():
    # Re-encode training data
    Zm, _ = encode_Y(vae, train_queue)
    Vt = vm(Dt, Wt).detach()
    Vtest = vm(Dtest, Wtest).detach()

    # GP prediction for test set
    vs = gp.get_vs()
    U, UBi, _ = gp.U_UBi_Shb([Vt], vs)
    Kiz = gp.solve(Zm, U, UBi, vs)
    Zo_test = vs[0] * Vtest.mm(Vt.transpose(0, 1).mm(Kiz))

    # Per-view test MSE
    test_Rid = test_data.Rid
    mse_per_view = {}
    mse_test_total = 0.0
    
    for data in test_queue:
        idxs = data[-1].to(device)
        Ytest = data[0].to(device)
        Yo = vae.decode(Zo_test[idxs])
        mse_batch = ((Ytest - Yo) ** 2).view(Ytest.shape[0], -1).mean(1)
        
        # Accumulate per-view
        for i, idx in enumerate(data[-1]):
            view = int(test_Rid[idx].item())
            if view not in mse_per_view:
                mse_per_view[view] = []
            mse_per_view[view].append(mse_batch[i].item())
        
        mse_test_total += mse_batch.sum().item()

    mse_test = mse_test_total / len(test_data)
    print(f"\n🎯 Test MSE (interpolation to unseen views): {mse_test:.6f}")
    print(f"\n📊 Test per-view MSE:")
    for view in sorted(mse_per_view.keys()):
        angle = int(view * 20)
        view_mse = np.mean(mse_per_view[view])
        print(f"   {angle:3d}°: {view_mse:.6f}")

## 14. View Results

In [ ]:
from IPython.display import Image, display
import glob

plot_files = sorted(glob.glob(os.path.join(fdir, "*.png")))
if plot_files:
    display(Image(filename=plot_files[-1]))

## 15. Download Results

In [ ]:
!zip -r /content/gppvae_periodic_interpolation_output.zip {CONFIG['outdir']}
from google.colab import files
files.download('/content/gppvae_periodic_interpolation_output.zip')